<a href="https://colab.research.google.com/github/m0tchy/camera-geometry-tutorial/blob/main/07_camera_pose(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備 (前回と同じ)

カメラの投影などを Matplotlib で工夫して表示する関数を定義する。
以下のセルを一度実行すること。

プロットではまず準備を行う関数 `init_3d_plot` を呼び、
続くプロットコマンドでは **Y, Z を逆にする**必要がある。

この関数は、 $z = 1$ の平面を正方形で表示する。
Z 上の位置は引数で変更できる。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def init_3d_plot(focal_length=1):

    # 投影面の定義
    screen_vertices = np.array([
        [-1, -1, focal_length], [1, -1, focal_length],
        [1, 1, focal_length], [-1, 1, focal_length]])
    screen_edges = [0, 1, 2, 3, 0]

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(projection="3d")
    ax.set_xlim(-5, 5)
    ax.set_ylim(-1, 9)  # Z 軸とみなす
    ax.set_zlim(5, -5)  # Y 軸とみなす
    ax.set_xlabel("X")
    ax.set_ylabel("Z")
    ax.set_zlabel("Y")
    ax.view_init(20, -25)

    # X, Z, Y の順で指定する
    ax.plot([0, 0], [-1, 9], [0, 0], ":", color="SILVER")
    ax.plot([0, 1], [0, 0], [0, 0], color="RED")    # X 軸
    ax.plot([0, 0], [0, 0], [0, 1], color="GREEN")  # Y 軸
    ax.plot([0, 0], [0, 1], [0, 0], color="BLUE")   # Z 軸

    ax.plot(screen_vertices[screen_edges, 0], screen_vertices[screen_edges, 2], screen_vertices[screen_edges, 1], color="GRAY")

    return fig, ax

# plt.figure などの代わりにこの関数を利用する
fig, ax = init_3d_plot()
# fig, ax = init_3d_plot(3)


# 復習：空間の点と辺の描画

3次元空間の点は、 4 次元ベクトルの同次座標で表現できる。
実際の座標は、第4成分で割った第1,2,3成分である。

Numpy 配列はインデックスが 0 から始まるので、成分の番号とのずれに注意。

In [ ]:
# 空間の点（同次座標） 複数の点を各行に並べる
X_ = np.array([
    [-1, -1, 5, 1], 
    [1, -1, 5, 1], 
    [1, -1, 7, 1], 
    [-1, -1, 7, 1], 

    [-1, 1, 5, 1],
    [1, 1, 5, 1],
    [1, 1, 7, 1],
    [-1, 1, 7, 1],

    [0, -2, 6, 1],
])

# 辺（折れ線）の集合: これは行列計算しないので、ただのリストでよい
edges = [
    [0,1,2,3,0], 
    [4,5,6,7,4], 
    [8,0,4], 
    [8,1,5], 
    [8,2,6], 
    [8,3,7], 
]

fig, ax = init_3d_plot()

# 普通の座標に変換
XX = X_[:, 0] / X_[:, 3]
XY = X_[:, 1] / X_[:, 3]
XZ = X_[:, 2] / X_[:, 3]
ax.scatter(XX, XZ, XY, color="BLACK")  # 表示のために Y, Z を入れ替える
for e in edges:
    ax.plot(XX[e], XZ[e], XY[e])  # 表示のために Y, Z を入れ替える

# 復習：透視投影

空間の点を投影中心に向かって投影面へ写すのを透視投影という。
投影面までの距離を $f$ とすると、その Z 座標が $f$ になればよいので、
$$
    \begin{align}
        X' &= (X / Z)f \\
        Y' &= (Y / Z)f \\
        Z' &= (Z / Z)f = f
    \end{align}
$$
となり、その同次座標は、
$$
    \begin{align}
    \begin{bmatrix} fX/Z \\ fY/Z \\ f \\ 1 \end{bmatrix}
    \sim 
    \begin{bmatrix} fX \\ fY \\ fZ \\ Z \end{bmatrix}
    \end{align}
$$
である。したがって、透視投影変換行列 $P_f$ を
$$
    P_f = \begin{bmatrix}
         f & 0 & 0 & 0 \\
         0 & f & 0 & 0 \\
         0 & 0 & f & 0 \\
         0 & 0 & 1 & 0
     \end{bmatrix}
$$
とすれば、 $Z = f$ 上への投影は、
$$
\tilde{\boldsymbol{X}'} \sim P_f \tilde{\boldsymbol{X}}
$$
となる。

以上をまとめると、このようになる。
$$
    \tilde{\boldsymbol{X}'}
    = \begin{bmatrix} X' \\ Y' \\ Z' \\ 1 \end{bmatrix}
    = \begin{bmatrix} fX/Z \\ fY/Z \\ f \\ 1 \end{bmatrix}
    \sim \begin{bmatrix} fX \\ fY \\ fZ \\ Z \end{bmatrix}
    = \begin{bmatrix}
         f & 0 & 0 & 0 \\
         0 & f & 0 & 0 \\
         0 & 0 & f & 0 \\
         0 & 0 & 1 & 0
    \end{bmatrix}
    \begin{bmatrix} X \\ Y \\ Z \\ 1 \end{bmatrix}
    = P_f\tilde{\boldsymbol{X}}
$$



In [ ]:
# distance to the screen from the origin
f = 2

projection = np.array([
    [f, 0, 0, 0], 
    [0, f, 0, 0], 
    [0, 0, f, 0], 
    [0, 0, 1, 0]])

# projected onto the screen
X2_ = (projection @ X_.T).T
print(X2_)

fig, ax = init_3d_plot(f)  # 引数でスクリーンまでの距離を指定する

# 光線を書く
ax.quiver(0, 0, 0, XX, XZ, XY, color="SILVER", linestyle="dashed", arrow_length_ratio=0)

# 普通の座標に変換
X2X = X2_[:, 0] / X2_[:, 3]
X2Y = X2_[:, 1] / X2_[:, 3]
X2Z = X2_[:, 2] / X2_[:, 3]

# 点を打つ
ax.scatter(XX, XZ, XY, color="BLACK")
ax.scatter(X2X, X2Z, X2Y, color="BLACK")

# 投影面上の図形

投影面上に描かれる図形について考えてみよう。

投影面上は、 Z が固定された X-Y 平面と考えることができるので、 Z 座標を無視して、平面プロットをすればよい。





In [ ]:
# 前の結果、 X2X, X2Y, X2Z を利用

# X2Z を使わないで、点を打つ
fig, ax = plt.subplots()
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.invert_yaxis()  # 画像の座標系になるように y 軸を下向きにする
ax.set_aspect("equal")

ax.scatter(X2X, X2Y, color="BLACK")
for e in edges:
    ax.plot(X2X[e], X2Y[e])  # 表示のために Y, Z を入れ替える

## 練習

この立体図形を、 Y 軸回りに回転させなさい。

# カメラの姿勢

ここまでは、カメラの投影中心が座標系の原点にあり、
投影面が Z 軸上に垂直に立っていることを仮定してきた。
この座標系はカメラ座標系と呼ばれている。


しかし、いろいろなシーンを CG でモデリングしたり、
実際に計測したりする場合にカメラの座標系での座標で作業するのは大変である。
それよりも、例えば部屋の床面を XY 平面とし部屋の角を原点としたり、
地球儀のように、緯度経度のような座標を使ったほうが何かと都合がよい。
このような物体のある空間の座標系を**世界座標系** またはワールド座標系 (world coordinate system) という。

また、カメラの位置を動かせるようにしたい。物体から離れたり、斜め上から撮影するようなことをしたい。
そこでカメラは世界座標系のどこかに投影中心があって、そこからカメラ座標系の向きが決められていると考えよう。
このカメラの「位置と向き」を合わせて**姿勢** (pose) という。

ここでの問題は、ある点の世界座標系での座標とカメラ座標系での座標の関係を知りたいということである。
座標系の区別をわかりやすくするために、それぞれ $\boldsymbol{X}^{\mathsf{W}}$, $\boldsymbol{X}^{\mathsf{C}}$ のように書くことにしよう。

2つの異なる座標系の間の変換を、座標変換という。
簡単のために、どちらも直交座標系であり、両者の単位（例えば [m] ）は一致していると仮定する。





## カメラの並進

今までの設定はカメラ座標系が世界座標系と一致していると考えられる。

そうすると、投影中心が世界座標で $\boldsymbol{C}$ にあるカメラは、カメラがそこへ平行移動したものである。
ここで、この座標はもちろん世界座標なので $\boldsymbol{C}^\mathsf{W}$ とするほうが統一がとれているが、紛らわしくない場合は省略する。


想像してほしい。カメラで見ている世界は、カメラが前に進むとどうなるだろうか？　その逆方向に移動することが分かるだろう。
したがって、向きの変わっていない単純に平行移動したカメラであれば、
\begin{align}
    \boldsymbol X^\mathsf{C} &= 
    \boldsymbol{X}^\mathsf{W} -\boldsymbol{C} \\
    \tilde{\boldsymbol X}^\mathsf{C} &= 
    \begin{bmatrix} 
        I_{3\times 3} & -\boldsymbol{C} \\
        \mathbf{0}_3^\top & 1 
    \end{bmatrix}
    \tilde{\boldsymbol{X}}^\mathsf{W}
\end{align}
というユークリッド変換をすることと同じである。
上が普通の座標での計算、下が同次座標での計算である。
これによって、世界座標をカメラ座標として読み直すことができる。
この行列を、カメラの姿勢行列とか外部パラメター行列という。

実際にやってみよう。
例えば、カメラの位置を後ろに下げると、投影された図形も小さくなる。

In [ ]:
# カメラの投影中心
# 原点より後ろに下げてみる
C = np.array([0, 0, -2])

# world to camera transfrom
camera_pose = np.array([
    [1, 0, 0, -C[0]],
    [0, 1, 0, -C[1]],
    [0, 0, 1, -C[2]],
    [0, 0, 0, 1],
])  # 逆平行移動

# 投影より前に座標変換を挟む
X2_ = (projection @ camera_pose @ X_.T).T

# 普通の座標に変換
X2X = X2_[:, 0] / X2_[:, 3]
X2Y = X2_[:, 1] / X2_[:, 3]
X2Z = X2_[:, 2] / X2_[:, 3]  # 2次元の表示には使わない

# X2Z を使わないで、点を打つ
fig, ax = plt.subplots()
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.invert_yaxis()  # 画像の座標系になるように y 軸を下向きにする
ax.set_aspect("equal")

ax.scatter(X2X, X2Y, color="BLACK")
for e in edges:
    ax.plot(X2X[e], X2Y[e])  # 表示のために Y, Z を入れ替える

## 練習

左右に移動させるとどうなるか試しなさい。
また、 3D でも表示してみなさい。

## カメラの向き

今度は左右に向きを変えてみよう。そうすると世界はその逆向きに回転する。
したがって、カメラの座標軸が世界座標からどう回転したかを $R$ と書けば、
座標変換はその逆変換 $R^{-1}$ となる。
一般に回転行列の逆は転置行列だったことを思い出そう。つまり、$R^{-1} = R^\top$ である。
したがって、座標系の向きが $R$ で回転している場合は、
$$
    \boldsymbol{X}^{\mathsf{C}} = 
    R^\top \boldsymbol{X}^{\mathsf{W}}
$$
$$
    \tilde{\boldsymbol{X}}^{\mathsf{C}} = 
    \begin{bmatrix} 
        R^\top & \boldsymbol{0} \\
        \mathbf{0}^\top & 1 
    \end{bmatrix}
    \tilde{\boldsymbol{X}}^{\mathsf{W}}
$$
となる。

例えばカメラが右を向くというのは、 Y 軸の正の回転である。これを確かめよう。右に 10 度カメラを回すと、物体は左に移動する。

In [ ]:
# カメラの投影中心は原点 (0, 0, 0)
# カメラが右を向く = Y 軸回りに回転
pan = np.radians(10)
rotation_pan = np.array([
    [np.cos(pan), 0, np.sin(pan), 0],
    [0, 1, 0, 0],
    [-np.sin(pan), 0, np.cos(pan), 0],
    [0, 0, 0, 1],
])  # Y 軸回りは他と違って下がマイナス

# world to camera transfrom
camera_pose = rotation_pan.T

# 投影より前に座標変換を挟む
X2_ = (projection @ camera_pose @ X_.T).T

# 普通の座標に変換
X2X = X2_[:, 0] / X2_[:, 3]
X2Y = X2_[:, 1] / X2_[:, 3]
X2Z = X2_[:, 2] / X2_[:, 3]  # 2次元の表示には使わない

# X2Z を使わないで、点を打つ
fig, ax = plt.subplots()
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.invert_yaxis()  # 画像の座標系になるように y 軸を下向きにする
ax.set_aspect("equal")

ax.scatter(X2X, X2Y, color="BLACK")
for e in edges:
    ax.plot(X2X[e], X2Y[e])  # 表示のために Y, Z を入れ替える

## 練習

1. このコードで定義している  Y 軸回りの回転ではカメラが左右に向きを変える。これを[**パン**](https://ja.wikipedia.org/wiki/%E3%83%91%E3%83%B3_(%E6%92%AE%E5%BD%B1%E6%8A%80%E6%B3%95)) (pan) とよぶ。この回転を $R_\text{pan}$ とすると、 $R^\top = R_\text{pan}^\top$ である。
2. 上下に動かしなさい。これを**チルト** (tilt) と呼ぶ。これは X 軸回りの回転である。 これを  $R^\top = R_\text{tilt}^\top$ と書くことにする。 
3. pan-tilt を同時に指定したい。それぞれの回転行列を連続して掛ければよいが、 tilt を先にしないと期待した結果にならない。つまり、 $R^\top = (R_\text{pan}R_\text{tilt})^\top$ とする。
これを確かめなさい。

* https://en.wikipedia.org/wiki/Panning_(camera)
* https://en.wikipedia.org/wiki/Tilt_(camera)

## カメラの姿勢

カメラの位置も向きも変わっている場合は、この2つを組み合わせればよい。
問題はその順番である。
回転はあくまでも原点中心であるため、先に向きを変えて平行移動する必要がある。そうすると、
$$
    \boldsymbol{X}^{\mathsf{C}} = 
    R^\top \boldsymbol{X}^{\mathsf{W}} - \boldsymbol{C}
$$
とすればよいと思うかもしれない。
しかし、これではだめである。
（以下の例で確認すること）

なぜなら、それぞれの計算はあくまでも世界座標に対して行われるものであるが、回転後の世界を平行移動するとき、その移動先も位置が変わっている。
新しい平行移動先は $R^\top\boldsymbol{C}$ である。
だから、正しい変換はこうなる。
$$
    \boldsymbol{X}^{\mathsf{C}} = 
    R^\top \boldsymbol{X}^{\mathsf{W}} - R^\top\boldsymbol{C}
$$
$$
    \tilde{\boldsymbol{X}}^{\mathsf{C}} = 
    \begin{bmatrix} 
        R^\top & -R^\top\boldsymbol{C} \\
        \mathbf{0}^\top & 1 
    \end{bmatrix}
    \tilde{\boldsymbol{X}}^{\mathsf{W}}
$$

これで世界座標系から、投影中心 $\boldsymbol{C}$ と向き $R$ の姿勢をもったカメラの座標系への変換を表す行列が定義できた。



## 補足

なお、
$$
    \boldsymbol{X}^{\mathsf{C}} = 
    R^\top(\boldsymbol{X}^{\mathsf{W}} - \boldsymbol{C})
$$
と書き直せるから（つまり並進してから回転）、
$$
    \tilde{\boldsymbol{X}}^{\mathsf{C}} = 
    \begin{bmatrix} 
        R^\top & \boldsymbol{0} \\
        \mathbf{0}^\top & 1 
    \end{bmatrix}
    \begin{bmatrix} 
        I & -\boldsymbol{C} \\
        \mathbf{0}^\top & 1 
    \end{bmatrix}
    \tilde{\boldsymbol{X}}^{\mathsf{W}}
$$
と分解することもできる。

コーディングする際はこちらのほうが書きやすいかもしれない。

In [ ]:
# カメラを前に移動し、左斜め上に移動する
# C = np.array([-2, -2, -2])
C = np.array([-2, -5, 2])

# カメラを右に 20 度、 下に 45 度下げる 
pan = np.radians(20)
tilt = np.radians(-45)

rotation_pan = np.array([
    [np.cos(pan), 0, np.sin(pan), 0],
    [0, 1, 0, 0],
    [-np.sin(pan), 0, np.cos(pan), 0],
    [0, 0, 0, 1],
])
rotation_tilt = np.array([
    [1, 0, 0, 0],
    [0, np.cos(tilt), -np.sin(tilt), 0],
    [0, np.sin(tilt), np.cos(tilt), 0],
    [0, 0, 0, 1],
])
R = rotation_pan @ rotation_tilt

translation_inv = np.array([
    [1, 0, 0, -C[0]],
    [0, 1, 0, -C[1]],
    [0, 0, 1, -C[2]],
    [0, 0, 0, 1],
])

# world to camera transfrom
# Y 軸回りは他と違って下がマイナスだが、逆回転なので上がマイナスである
camera_pose = R.T @ translation_inv

# 投影より前に座標変換を挟む
X2_ = (projection @ camera_pose @ X_.T).T

# 普通の座標に変換
X2X = X2_[:, 0] / X2_[:, 3]
X2Y = X2_[:, 1] / X2_[:, 3]
X2Z = X2_[:, 2] / X2_[:, 3]  # 2次元の表示には使わない

# X2Z を使わないで、点を打つ
fig, ax = plt.subplots()
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.invert_yaxis()  # 画像の座標系になるように y 軸を下向きにする
ax.set_aspect("equal")

ax.scatter(X2X, X2Y, color="BLACK")
for e in edges:
    ax.plot(X2X[e], X2Y[e])  # 表示のために Y, Z を入れ替える

## 点の変換と座標変換のまとめ

これまで、平面や空間の点を**動かす**変換をいろいろ扱ったが、
今回のように点は動かずに座標を決める**座標系を変える**変換でも計算上は単に線形変換の行列をかけたり平行移動のためのベクトルを足すだけであり、解釈の違いでしかないことが分かる。

https://en.wikipedia.org/wiki/Active_and_passive_transformation

一般に、物体をカメラで撮影するとき、以下のような式になる。

\begin{align}
\tilde{\boldsymbol{X'}} &\sim P_f M \tilde{\boldsymbol{X}}^\mathsf{W} \\
 &\sim P_f \tilde{\boldsymbol{X}}^\mathsf{C}
\end{align}

ここで、 
* $M$ はカメラの姿勢を表す $\mathsf{W}\to\mathsf{C}$ の座標変換である。これを外部パラメターという。また、ビュー (view) 行列ということもある。
* $P_f$ は投影面への透視投影 (perspective projection) である。（なお、ここではまだカメラ座標系だが、今後画像座標系への変換に拡張される。）

さて、今回の家の形のオブジェクトは世界座標で直接頂点の場所が指定されてる。
しかし、オブジェクトはいろいろな位置や大きさでコピーを配置できるようにできたほうが便利である。そこで、世界座標とは別にモデル独自の座標系でデザインをしたほうがやりやすい。
例えば家の例の場合は、床面の中央に原点があり、そこから相対的に点の位置を指定する。
そして、シーンに置くときにこれまでやったようなユークリッド変換（もしくは拡大縮小などを含むアフィン変換）を適応して配置するのである。

これを含めると変換はこのようになる。

\begin{align}
\tilde{\boldsymbol{X'}} &\sim P_f M_\text{view} M_\text{model} \tilde{\boldsymbol{X}}^\mathsf{M} \\
 &\sim P_f M_\text{view} \tilde{\boldsymbol{X}}^\mathsf{W} \\
 &\sim P_f \tilde{\boldsymbol{X}}^\mathsf{C}
\end{align}

* $M_\text{model}$ は物体を配置するために変換するための行列である。物体は独自のモデル座標系 $\mathsf{M}$ で記述されていると考えて、それをワールド座標系 $\mathsf{W}$ に変換している。
* OpenGL というグラフィックスのライブラリでは $M = M_\text{view} M_\text{model}$ を**モデル・ビュー**変換として１つにまとめて扱っている。


## 練習
これまでの例の家のモデル `X_` と `edges` を床面の中央を原点としたものに書き直し、モデルの変換行列を定義して元のものと同じ位置になるようにする（つまり、原点を (0,0,6) に移動させる変換）。
投影変換の際にモデル変換をかけて表示すればこれまでの例と同じになる。

In [ ]:
# 解答例

# (0, 0, 0) が床面の中央なっている家のモデル
X_ = np.array([
    [-1, -1, -1, 1], 
    [1, -1, -1, 1], 
    [1, -1, 1, 1], 
    [-1, -1, 1, 1], 

    [-1, 1, -1, 1],
    [1, 1, -1, 1],
    [1, 1, 1, 1],
    [-1, 1, 1, 1],

    [0, -2, 0, 1],
])

edges = [
    [0,1,2,3,0], 
    [4,5,6,7,4], 
    [8,0,4], 
    [8,1,5], 
    [8,2,6], 
    [8,3,7], 
]

# Z = 6 まで平行移動して配置する
model_transform = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 6],
    [0, 0, 0, 1],
])

# X はモデル変換・ビュー変換・透視投影の順で変換される
X2_ = (projection @ camera_pose @ model_transform @ X_.T).T

# 普通の座標に変換
X2X = X2_[:, 0] / X2_[:, 3]
X2Y = X2_[:, 1] / X2_[:, 3]
X2Z = X2_[:, 2] / X2_[:, 3]  # 2次元の表示には使わない

# X2Z を使わないで、点を打つ
fig, ax = plt.subplots()
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.invert_yaxis()  # 画像の座標系になるように y 軸を下向きにする
ax.set_aspect("equal")

ax.scatter(X2X, X2Y, color="BLACK")
for e in edges:
    ax.plot(X2X[e], X2Y[e])  # 表示のために Y, Z を入れ替える